In [1]:
!huggingface-cli login
!pip install -U bitsandbytes


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `LLM` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `LLM`
   ━━━━━━

In [2]:
!unzip /content/lora-llama3.2-python-chkpoint.zip -d /content/lora-llama3.2-python-chkpoint
!unzip /content/lora-llama3.2-python-tokens.zip -d /content/lora-llama3.2-python-tokens

Archive:  /content/lora-llama3.2-python-chkpoint.zip
  inflating: /content/lora-llama3.2-python-chkpoint/adapter_model.safetensors  
  inflating: /content/lora-llama3.2-python-chkpoint/adapter_config.json  
  inflating: /content/lora-llama3.2-python-chkpoint/README.md  
Archive:  /content/lora-llama3.2-python-tokens.zip
  inflating: /content/lora-llama3.2-python-tokens/special_tokens_map.json  
  inflating: /content/lora-llama3.2-python-tokens/tokenizer_config.json  
  inflating: /content/lora-llama3.2-python-tokens/tokenizer.json  


In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM, TextStreamer
from peft import PeftModel
import torch

In [4]:
# Load tokenizer and base model
tokenizer = AutoTokenizer.from_pretrained("/content/lora-llama3.2-python-tokens")
base_model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-1B",  # or your base model name
    load_in_8bit=True,           # or False depending on what you used
    device_map="auto"
)






/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/843 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


model.safetensors:   0%|          | 0.00/2.47G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/185 [00:00<?, ?B/s]

In [5]:
# Load LoRA adapter on top of base model
# model = PeftModel.from_pretrained(base_model, "/content/lora-llama3.2-python-chkpoint/checkpoint-1125") # in cse of QLORA
model = PeftModel.from_pretrained(base_model, "/content/lora-llama3.2-python-chkpoint") # in cse of LORA
model.eval()

/usr/local/lib/python3.11/dist-packages/peft/config.py:162: UserWarning: Unexpected keyword arguments ['corda_config', 'trainable_token_indices'] for class LoraConfig, these are ignored. This probably means that you're loading a configuration file that was saved using a higher version of the library and additional parameters have been introduced since. It is highly recommended to upgrade the PEFT version before continuing (e.g. by running `pip install -U peft`).
  warnings.warn(


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): LlamaForCausalLM(
      (model): LlamaModel(
        (embed_tokens): Embedding(128256, 2048)
        (layers): ModuleList(
          (0-15): 16 x LlamaDecoderLayer(
            (self_attn): LlamaAttention(
              (q_proj): lora.Linear8bitLt(
                (base_layer): Linear8bitLt(in_features=2048, out_features=2048, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=2048, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=8, out_features=2048, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): L

In [10]:
# Optional: streamer to see output as it generates
streamer = TextStreamer(tokenizer)

# Input prompt
prompt = "Write a Python function to check if a number is prime."

# Tokenize
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate output
with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=150,
        do_sample=True,
        temperature=0.7,
        top_p=0.95,
        pad_token_id=tokenizer.eos_token_id  # Suppress pad_token_id warning
        # streamer=streamer  # Optional: use if you want to stream output
    )



In [11]:
# Decode output
generated_tokens = outputs[0][inputs['input_ids'].shape[1]:]  # Skip input tokens
generated_text = tokenizer.decode(generated_tokens, skip_special_tokens=True)
print("\nGenerated Output:\n", generated_text)


Generated Output:
  A prime number is a natural number greater than 1 that is divisible only by 1 and itself.
def isPrime(n):
    if n <= 1:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
    return True
